In [23]:
import pandas as pd
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms

In [2]:
botswana = pd.read_csv('botswana.tsv', sep='\t')
botswana.head()

,ceb,age,educ,religion,idlnchld,knowmeth,usemeth,evermarr,agefm,heduc,urban,electric,radio,tv,bicycle
0,0,18,10,catholic,4.0,1.0,1.0,0,NaN,NaN,1,1.0,1.0,1.0,1.0
1,2,43,11,protestant,2.0,1.0,1.0,1,20.0,14.0,1,1.0,1.0,1.0,1.0
2,0,49,4,spirit,4.0,1.0,0.0,1,22.0,1.0,1,1.0,1.0,0.0,0.0
3,0,24,12,other,2.0,1.0,0.0,0,NaN,NaN,1,1.0,1.0,1.0,1.0
4,3,32,13,other,3.0,1.0,1.0,1,24.0,12.0,1,1.0,1.0,1.0,1.0


### 1

In [3]:
botswana['religion'].nunique()

4

### 2

In [4]:
botswana.dropna().shape[0]

1834

### 3

In [5]:
botswana['nevermarr'] = 0
botswana.loc[botswana['agefm'].isna(), 'nevermarr'] = 1
botswana.drop('evermarr', axis=1, inplace=True)
botswana['agefm'].fillna(0, inplace=True)
botswana.loc[botswana['nevermarr'] == 1, 'heduc'] = -1
botswana.head()

,ceb,age,educ,religion,idlnchld,knowmeth,usemeth,agefm,heduc,urban,electric,radio,tv,bicycle,nevermarr
0,0,18,10,catholic,4.0,1.0,1.0,0.0,-1.0,1,1.0,1.0,1.0,1.0,1
1,2,43,11,protestant,2.0,1.0,1.0,20.0,14.0,1,1.0,1.0,1.0,1.0,0
2,0,49,4,spirit,4.0,1.0,0.0,22.0,1.0,1,1.0,1.0,0.0,0.0,0
3,0,24,12,other,2.0,1.0,0.0,0.0,-1.0,1,1.0,1.0,1.0,1.0,1
4,3,32,13,other,3.0,1.0,1.0,24.0,12.0,1,1.0,1.0,1.0,1.0,0


In [6]:
botswana['heduc'].isna().sum()

123

### 4

In [7]:
for c, v in zip(['idlnchld', 'heduc', 'usemeth'], [-1, -2, -1]):
    botswana[c+'_noans'] = 0
    botswana.loc[botswana[c].isna(), c+'_noans'] = 1
    botswana.loc[botswana[c].isna(), c] = v
botswana.head()

,ceb,age,educ,religion,idlnchld,knowmeth,usemeth,agefm,heduc,urban,electric,radio,tv,bicycle,nevermarr,idlnchld_noans,heduc_noans,usemeth_noans
0,0,18,10,catholic,4.0,1.0,1.0,0.0,-1.0,1,1.0,1.0,1.0,1.0,1,0,0,0
1,2,43,11,protestant,2.0,1.0,1.0,20.0,14.0,1,1.0,1.0,1.0,1.0,0,0,0,0
2,0,49,4,spirit,4.0,1.0,0.0,22.0,1.0,1,1.0,1.0,0.0,0.0,0,0,0,0
3,0,24,12,other,2.0,1.0,0.0,0.0,-1.0,1,1.0,1.0,1.0,1.0,1,0,0,0
4,3,32,13,other,3.0,1.0,1.0,24.0,12.0,1,1.0,1.0,1.0,1.0,0,0,0,0


In [9]:
botswana.dropna(inplace=True)

In [14]:
r, c = botswana.shape
r*c

78264

### 5-6

In [24]:
m1 = smf.ols('ceb ~ age + educ + religion + idlnchld + knowmeth + usemeth + agefm + heduc + urban + electric + '\
             'radio + tv + bicycle + nevermarr + idlnchld_noans + heduc_noans + usemeth_noans', data=botswana)
fitted1 = m1.fit()
print(fitted1.summary())

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     412.5
Date:                Sun, 08 Aug 2021   Prob (F-statistic):               0.00
Time:                        15:59:01   Log-Likelihood:                -7732.1
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4328   BIC:                         1.563e+04
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                 -1

### 7

In [26]:
sms.het_breuschpagan(fitted1.resid, fitted1.model.exog)[1]

1.1452927633439797e-225

In [27]:
m2 = smf.ols('ceb ~ age + educ + religion + idlnchld + knowmeth + usemeth + agefm + heduc + urban + electric + '\
             'radio + tv + bicycle + nevermarr + idlnchld_noans + heduc_noans + usemeth_noans', data=botswana)
fitted2 = m2.fit(cov_type='HC1')
print(fitted2.summary())

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     345.0
Date:                Sun, 08 Aug 2021   Prob (F-statistic):               0.00
Time:                        16:01:57   Log-Likelihood:                -7732.1
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4328   BIC:                         1.563e+04
Df Model:                          19                                         
Covariance Type:                  HC1                                         
                             coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                 -1

### 8

In [28]:
m3 = smf.ols('ceb ~ age + educ + idlnchld + knowmeth + usemeth + agefm + heduc + urban + electric + '\
             'bicycle + nevermarr + idlnchld_noans + heduc_noans + usemeth_noans', data=botswana)
fitted3 = m3.fit()
print(fitted3.summary())

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     559.5
Date:                Sun, 08 Aug 2021   Prob (F-statistic):               0.00
Time:                        16:03:24   Log-Likelihood:                -7734.5
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4333   BIC:                         1.559e+04
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         -1.0698      0.198     -5.

In [29]:
sms.het_breuschpagan(fitted3.resid, fitted3.model.exog)[1]

1.1197458896536614e-228

In [30]:
m4 = smf.ols('ceb ~ age + educ + idlnchld + knowmeth + usemeth + agefm + heduc + urban + electric + '\
             'bicycle + nevermarr + idlnchld_noans + heduc_noans + usemeth_noans', data=botswana)
fitted4 = m4.fit(cov_type='HC1')
print(fitted4.summary())

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     463.4
Date:                Sun, 08 Aug 2021   Prob (F-statistic):               0.00
Time:                        16:04:12   Log-Likelihood:                -7734.5
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4333   BIC:                         1.559e+04
Df Model:                          14                                         
Covariance Type:                  HC1                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         -1.0698      0.258     -4.

In [37]:
p = fitted1.compare_f_test(fitted3)[1]
round(p, 4)

0.4672

### 9

In [38]:
m5 = smf.ols('ceb ~ age + educ + idlnchld + knowmeth + agefm + heduc + urban + electric + '\
             'bicycle + nevermarr + idlnchld_noans + heduc_noans', data=botswana)
p = fitted3.compare_f_test(m5.fit())[1]
print(p)

3.1552009480442536e-40
